In [2]:
import os
import pandas as pd
import sqlalchemy

from dotenv import load_dotenv

# load environment variables
load_dotenv()

True

In [3]:
def connect_engine(url):
    engine = sqlalchemy.create_engine(url)
    return engine

def get_data(sql, engine):
    
    df = pd.read_sql(sql, con=engine)
    return df

In [4]:
connection_url = f"postgresql://{os.getenv('POSTGRESQL_USER')}:{os.getenv('POSTGRESQL_PW')}@{os.getenv('POSTGRESQL_IP')}/{os.getenv('POSTGRESQL_DB')}"

engine = connect_engine(connection_url)
sql_vault = """
    select collateral_ratio, collateral_value, loan_value
    from vaults
    order by created_at desc limit 1;
    """
df = get_data(sql_vault, engine)

In [5]:
df.iloc[0,1]

36227.85

In [6]:
# query to get current token holdings in the wallet
sql_tokens = """
    select distinct on (dt1.token_id) dt1.token_id, dt1.symbol as token_symbol, dt1.islps, amount, dt2.symbol as tokena_symbol, dt3.symbol as tokenb_symbol, tokena_reserve, tokenb_reserve, total_liquidity_token
    from defichain_tokens dt1
    left join defichain_tokens dt2 on dt1.tokena_id=dt2.token_id
    left join defichain_tokens dt3 on dt1.tokenb_id=dt3.token_id
    inner join defichain_holdings dh on dh.token_id=dt1.token_id
    order by dt1.token_id, dh.created_at desc;
    """

df_tokens = get_data(sql_tokens, engine)
df_tokens["tokena_amount"] = df_tokens["amount"] / df_tokens["total_liquidity_token"] * df_tokens["tokena_reserve"]
df_tokens["tokenb_amount"] = df_tokens["amount"] / df_tokens["total_liquidity_token"] * df_tokens["tokenb_reserve"]
df_tokena = df_tokens[["tokena_symbol", "tokena_amount"]].groupby('tokena_symbol').sum().reset_index().rename(columns={'tokena_symbol': 'symbol', 'tokena_amount': 'amount'})
df_tokenb = df_tokens[["tokenb_symbol", "tokenb_amount"]].groupby('tokenb_symbol').sum().reset_index().rename(columns={'tokenb_symbol': 'symbol', 'tokenb_amount': 'amount'})
df_token_wallet = df_tokens[df_tokens["islps"] == False][["token_symbol", "amount"]].rename(columns={'token_symbol': 'symbol'})
df_token_all = pd.concat([df_tokena, df_tokenb, df_token_wallet]).groupby('symbol').sum()
for idx in df_token_all.index:
    print(idx)

AAPL
CS
DFI
DUSD
LTC
NVDA
PLTR


In [7]:
sql_vault = """
    with vault_CTE (id, created_at, collateral_ratio, collateral_value, loan_value)
    as
    (
        select id, created_at, collateral_ratio, collateral_value, loan_value
        from vaults
        order by created_at desc limit 1
    )
    select id, vault_CTE.created_at, vault_amounts.token_id, name, symbol, token_type, amount, active_price, next_price
    from vault_CTE
    inner join vault_amounts on vault_CTE.id = vault_amounts.vault_id
    inner join defichain_tokens on vault_amounts.token_id=defichain_tokens.token_id;
    """
df_vault = get_data(sql_vault, engine)
active_dfi_price = df_vault[df_vault["token_id"] == 0].active_price.iloc[0]
active_dfi_price
# df["active_value"] = df["amount"] * df["active_price"]
# df["next_value"] = df["amount"] * df["next_price"]
# df_vault_details = df.groupby("token_type").sum()
# df_vault_details.reset_index(inplace=True)
# print(df_vault_details[df_vault_details["token_type"] == 'collateral'].next_value.iloc[0] - df_vault_details[df_vault_details["token_type"] == 'collateral'].active_value.iloc[0])
# df_vault_details

0.48512864

In [8]:
df_coll = df_vault[df_vault["token_type"] == 'collateral']
df_coll

,id,created_at,token_id,name,symbol,token_type,amount,active_price,next_price
0,2320,2022-12-05 19:30:04.449693+00:00,0,Default Defi token,DFI,collateral,32196.664,0.485129,0.485282
1,2320,2022-12-05 19:30:04.449693+00:00,15,Decentralized USD,DUSD,collateral,17173.607,1.000000,1.000000


In [9]:
# query to get 24 hours worth of DFI dex prices
from datetime import datetime, timedelta
import pytz
# query to get 24 hours worth of DFI dex prices

d = (datetime.now(pytz.utc) - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
print(d)
sql_dfi_prices = f"""
    select created_at, active_price
    from vault_amounts
    where token_id = 0 and created_at > '{d}'
    order by created_at desc;
    """
df_dfi_dex_prices_24h = get_data(sql_dfi_prices, engine)
df_dfi_dex_prices_24h.active_price.iloc[-1]

2022-12-04 19:35:06


0.4770983